# Creación y carga de base de datos en PostgreSQL desde Python

In [1]:
# Importar paquetes
import pandas as pd
import psycopg2

In [2]:
df = pd.read_excel("../data/datos_churn_limpios.xlsx") # Carga de datos

## 1. Crear la base de datos en PostgreSQL 

In [ ]:
# Una vez diseñado el modelo relacional, crear el script SQL para crear las tablas de este en postgreSQL, validando que coincidan los tipos de datos con el dataset 
'''
CREATE TABLE clientes (
    customer_id TEXT PRIMARY KEY,
    gender TEXT,
    age INT,
    under_30 TEXT,
    senior_65 TEXT,
    group_name TEXT,
    state_name TEXT,
    customer_segment TEXT
);

CREATE TABLE contratos (
    id_contrato SERIAL PRIMARY KEY,
    customer_id TEXT REFERENCES clientes(customer_id),
    contract_type TEXT,
    payment_method TEXT,
    applied_discount TEXT,
    unlimited_data_plan TEXT,
    device_protection_online_backup TEXT,
    number_of_customers_in_group INT
);

CREATE TABLE consumo (
    id_consumo SERIAL PRIMARY KEY,
    customer_id TEXT REFERENCES clientes(customer_id),
    avg_monthly_gb_download INT,
    extra_data_charges INT,
    monthly_charge INT,
    total_charges FLOAT,
    average_monthly_expenses FLOAT
);

CREATE TABLE soporte (
    id_soporte SERIAL PRIMARY KEY,
    customer_id TEXT REFERENCES clientes(customer_id),
    customer_service_calls INT,
    number_of_complaints_or_support INT,
    preferred_contact_method TEXT
);

CREATE TABLE historial (
    id_historial SERIAL PRIMARY KEY,
    customer_id TEXT REFERENCES clientes(customer_id),
    account_length_in_months INT,
    customer_tenure_in_months INT,
    contact_date TIMESTAMP,
    last_transaction_date TIMESTAMP,
    year_name INT,
    month_name INT,
    quarter INT
);

CREATE TABLE churn (
    id_churn SERIAL PRIMARY KEY,
    customer_id TEXT REFERENCES clientes(customer_id),
    churn_label TEXT,
    churn_category TEXT,
    churn_reason TEXT
);
'''

## 2. Importar los datos desde Python a PostgreSQL

#### Dividir el dataframe original en dataframes por tabla según el modelo relacional creado

In [4]:
# Renombrar columnas problemáticas para que coincidan con PostgreSQL
df = df.rename(columns={
    'group': 'group_name',
    'year': 'year_name',
    'state': 'state_name',
    'month': 'month_name',
    'device_protection_&_online_backup':'device_protection_online_backup'
})

# Dividir el dataframe en sub-dataframes según el modelo relacional

# Tabla: clientes
df_clientes = df[[
    'customer_id', 'gender', 'age', 'under_30', 'senior_65',
    'group_name', 'state_name', 'customer_segment'
]].drop_duplicates()

# Tabla: contratos
df_contratos = df[[
    'customer_id', 'contract_type', 'payment_method',
    'applied_discount', 'unlimited_data_plan',
    'device_protection_online_backup', 'number_of_customers_in_group'
]]

# Tabla: consumo
df_consumo = df[[
    'customer_id', 'avg_monthly_gb_download', 'extra_data_charges',
    'monthly_charge', 'total_charges', 'average_monthly_expenses'
]]

# Tabla: soporte
df_soporte = df[[
    'customer_id', 'customer_service_calls',
    'number_of_complaints_or_support', 'preferred_contact_method'
]]

# Tabla: historial
df_historial = df[[
    'customer_id', 'account_length_(in_months)',
    'customer_tenure_(in_months)', 'contact_date',
    'last_transaction_date', 'year_name', 'month_name', 'quarter'
]]
# Renombrar columnas para que coincidan con las de PostgreSQL
df_historial.columns = [
    'customer_id', 'account_length_in_months',
    'customer_tenure_in_months', 'contact_date',
    'last_transaction_date', 'year_name', 'month_name', 'quarter'
]

# Tabla: churn
df_churn = df[[
    'customer_id', 'churn_label', 'churn_category', 'churn_reason'
]]

# Mostrar shapes para confirmar
(df_clientes.shape, df_contratos.shape, df_consumo.shape, df_soporte.shape,
 df_historial.shape, df_churn.shape)


((6687, 8), (6690, 7), (6690, 6), (6690, 4), (6690, 8), (6690, 4))

## Insertar todas las tablas a PostgreSQL

### 1. Conexión con psycopg2

In [5]:
# Conexión a PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    dbname="proyecto_churn_amazon",
    user="postgres",
    password="admin",
    port=5432
)

### 2. Función para insertar un DataFrame en una tabla

In [6]:
def insertar_dataframe(df, tabla_sql, columnas_sql, conn):
    cursor = conn.cursor()
    for _, fila in df.iterrows():
        valores = tuple(fila[col] for col in columnas_sql)
        placeholders = ','.join(['%s'] * len(valores))
        query = f"INSERT INTO {tabla_sql} ({','.join(columnas_sql)}) VALUES ({placeholders})"
        cursor.execute(query, valores)
    conn.commit()
    print(f"✅ Datos insertados en {tabla_sql}")

### 3. Insertar los datos en cada tabla de PostgreSQL

In [ ]:
insertar_dataframe(df_clientes, "clientes", [
    "customer_id", "gender", "age", "under_30", "senior_65",
    "group_name", "state_name", "customer_segment"
], conn)

insertar_dataframe(df_contratos, "contratos", [
    "customer_id", "contract_type", "payment_method",
    "applied_discount", "unlimited_data_plan",
    "device_protection_online_backup", "number_of_customers_in_group"
], conn)

insertar_dataframe(df_consumo, "consumo", [
    "customer_id", "avg_monthly_gb_download", "extra_data_charges",
    "monthly_charge", "total_charges", "average_monthly_expenses"
], conn)

insertar_dataframe(df_soporte, "soporte", [
    "customer_id", "customer_service_calls",
    "number_of_complaints_or_support", "preferred_contact_method"
], conn)

insertar_dataframe(df_historial, "historial", [
    "customer_id", "account_length_in_months",
    "customer_tenure_in_months", "contact_date",
    "last_transaction_date", "year_name", "month_name", "quarter"
], conn)

insertar_dataframe(df_churn, "churn", [
    "customer_id", "churn_label", "churn_category", "churn_reason"
], conn)

# Cerrar la conexión
conn.close()

### Documentación: Integración del proyecto con PostgreSQL (Modelo Relacional + Python + SQL)

Para profesionalizar aún más el proyecto y demostrar dominio técnico, se ha construido una **base de datos relacional en PostgreSQL** a partir del dataset de atención al cliente. A continuación se detallan los pasos realizados:

---

#### 1. Diseño del modelo relacional

Se dividió el dataset original en seis tablas relacionadas (`clientes`,`contratos`,`consumo`,`historial`,`soporte`,`churn`)

Se definieron **claves primarias** y **foráneas** para mantener la integridad referencial entre las tablas.

---

#### 2. Creación de las tablas en PostgreSQL

Desde pgAdmin se creó una nueva base de datos llamada `proyecto_churn_amazon`, y se ejecutaron scripts SQL para crear las tablas con las estructuras correctas, incluyendo las claves necesarias para modelar bien las relaciones entre entidades.

---

#### 3. Preparación de los datos en Python

- Se cargó el dataset limpio
- Se separaron los datos en 6 DataFrames (`clientes`,`contratos`,`consumo`,`historial`,`soporte`,`churn`)
- Se asignaron identificadores únicos (`customer_id`, `id_contrato`, `id_consumo`,`id_historial`,`id_soporte`,`id_churn`)
- Se renombraron las columnas para que coincidan con las definidas en SQL
- Se dejó lista cada tabla para ser insertada correctamente en la base de datos

---

#### 4. Conexión a PostgreSQL desde Python

Se utilizó la librería `psycopg2` para conectarse a la base y realizar inserciones:

- Se definió una función general `insertar_dataframe()` para insertar cualquier tabla
- Se insertaron los datos en el siguiente orden: (`clientes`,`contratos`,`consumo`,`soporte`,`historial`,`churn`)
- Se verificó que los datos quedaron cargados correctamente

---

#### Resultado

Ya está disponible en PostgreSQL una base de datos relacional realista, funcional y lista para:

- Realizar **consultas SQL** para obtener insights
- Ser utilizada desde Python con `psycopg2` o `SQLAlchemy`
- Conectarla desde Power BI para crear dashboards optimizados y escalables

Este trabajo demuestra no solo habilidades en análisis de datos, sino también en **modelado de bases de datos**, **ETL básico**, **SQL relacional** y buenas prácticas para entornos de producción.

---------
# Consultas SQL ejecutadas desde Python orientadas al análisis del negocio

En esta sección se utilizarán **consultas SQL** sobre la base de datos relacional creada en PostgreSQL para:

- Reproducir los **insights clave del análisis exploratorio** realizado previamente en Python
- Demostrar **dominio del lenguaje SQL** para responder preguntas reales del negocio
- Validar que la estructura relacional permite extraer información útil de forma eficiente
- Potenciar el uso de SQL como herramienta fundamental del analista de datos

Las consultas están orientadas a temas como:

- Identificación de cuellos de botella por agente o canal
- Análisis de tiempos de respuesta y satisfacción
- Evaluación del servicio ofrecido a clientes VIP
- Control de tickets de alta prioridad y su resolución
- Distribución de carga horaria y por categoría

Cada consulta responderá a una pregunta de negocio específica, y permitirá mostrar cómo se puede utilizar SQL para obtener valor directamente desde la base de datos, sin necesidad de otras herramientas.

In [ ]:
# Conexión a PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    dbname="proyecto_churn_amazon",
    user="postgres",
    password="admin",
    port=5432
)
cursor = conn.cursor()
# Crear una función para ejecutar consultas SQL
def ejecutar_consulta(sql):
    return pd.read_sql_query(sql, conn)

Consulta 1: ¿Los clientes con descuentos aplicados abandonan menos que los que no tienen descuento?

In [ ]:
sql = """
SELECT 
  applied_discount,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY applied_discount;
"""
ejecutar_consulta(sql)
# Objetivo:  Evaluar si las promociones o descuentos están cumpliendo su objetivo de fidelizar a los clientes, o si se está perdiendo dinero en estrategias que no reducen el abandono.

Consulta 2: ¿Cuál es la tasa de churn según el tipo de contrato?

In [ ]:
sql = """
SELECT 
  c.contract_type,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY c.contract_type;

"""
ejecutar_consulta(sql)
# Objetivo: Identificar si los contratos mensuales tienen mayor abandono que los de largo plazo.

Consulta 3: ¿Los clientes con plan familiar abandonan menos que los individuales?

In [ ]:
sql = """
SELECT 
  number_of_customers_in_group,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY number_of_customers_in_group
ORDER BY number_of_customers_in_group;
"""
ejecutar_consulta(sql)
# Objetivo: Evaluar si los planes compartidos ayudan a reducir el churn.

Consulta 4: ¿Qué tasa de churn presentan los clientes menores de 30 años frente a los mayores de 65?

In [ ]:
sql = """
SELECT 
  cl.under_30,
  cl.senior_65,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM clientes cl
JOIN churn ch USING (customer_id)
GROUP BY cl.under_30, cl.senior_65;
"""
ejecutar_consulta(sql)
# Objetivo: Comparar la fidelidad generacional de los clientes.

Consulta 5:  ¿Qué impacto tiene el número de llamadas al soporte sobre el churn?

In [ ]:
sql = """
SELECT 
  customer_service_calls,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM soporte s
JOIN churn ch USING (customer_id)
GROUP BY customer_service_calls
ORDER BY customer_service_calls;
"""
ejecutar_consulta(sql)
# Objetivo: Evaluar si múltiples llamadas predicen abandono por mal servicio.

Consulta 6:  ¿Cómo varía el churn según si el cliente tiene o no un plan de datos ilimitado?

In [ ]:
sql = """
SELECT 
  unlimited_data_plan,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY unlimited_data_plan;
"""
ejecutar_consulta(sql)
# Objetivo: Analizar si tener datos ilimitados influye realmente en la retención.

Consulta 7:  ¿Los clientes con protección de dispositivos y backup abandonan menos?

In [ ]:
sql = """
SELECT 
  device_protection_and_backup,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY device_protection_and_backup;
"""
ejecutar_consulta(sql)
# Objetivo:  Validar si los servicios extra aumentan la lealtad.

Consulta 8: ¿Cuál es la tasa de churn según el método de pago?

In [ ]:
sql = """
SELECT 
  payment_method,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM contratos c
JOIN churn ch USING (customer_id)
GROUP BY payment_method;
"""
ejecutar_consulta(sql)
# Objetivo: Detectar si los pagos manuales generan más abandono que los automáticos.

Consulta 9: ¿Cuál es la tasa de churn en función del tiempo que lleva el cliente en la empresa?

In [ ]:
sql = """
SELECT 
  customer_tenure_in_months,
  ROUND(100.0 * COUNT(*) FILTER (WHERE ch.churn_label = 'yes') / COUNT(*), 2) AS tasa_churn
FROM historial h
JOIN churn ch USING (customer_id)
GROUP BY customer_tenure_in_months
ORDER BY customer_tenure_in_months;
"""
ejecutar_consulta(sql)
# Objetivo: Ver si los clientes nuevos abandonan más que los de larga duración.

Consulta 10: ¿Qué razones de abandono son más frecuentes entre los clientes que han hecho múltiples contactos con soporte?

In [ ]:
sql = """
SELECT 
  ch.churn_reason,
  COUNT(*) AS num_clientes
FROM soporte s
JOIN churn ch USING (customer_id)
WHERE s.number_of_complaints_or_support >= 3
  AND ch.churn_label = 'yes'
GROUP BY ch.churn_reason
ORDER BY num_clientes DESC;
"""
ejecutar_consulta(sql)
# Objetivo: Conocer las causas de abandono más comunes tras experiencias negativas con el soporte.